In [1]:
#Fetch Gemini API Key
import os
from getpass import getpass

os.environ["GOOGLE_API_KEY"] = getpass("Gemini API Key:")

**bbc** - 크롤링 불가, 유기 <br>
**블랙월** - 광고 너무 많아서 유기, 요약문만 사용? <br>
**뉴욕타임즈** - API사용, 요약본, URL, 타이틀<br>
**CNN** - 전문 긁어오기

In [2]:
import pandas as pd

# 기사 데이터 수집

In [92]:
# cnn 기사 데이터 수집
# cnn_news.csv 파일 경로 수정
cnn=pd.read_csv("../data/cnn_news.csv")

# 기사 선정 기준? 
# articles로 model/chatbots에서 날아옴, 수정해야함
import random
while True: 
    i=int(random.randint(0,len(cnn)))
    # article=cnn.iloc[i]
    article=cnn.iloc[3]
    if not (article.content=='' or article.content=="No content available."): break
article



title      Grammys and Oscars to proceed amid LA fires, b...
link       https://edition.cnn.com/2025/01/13/entertainme...
content    As they say in Hollywood, the show must go on....
Name: 3, dtype: object

In [19]:
print(article.content)

Every time a celebrity poses on a red carpet, countless cameras flash, forever immortalizing their outfit, preserving the hundreds or even thousands of hours it has taken to create.

Sunday night’s Golden Globes were no different, with Zendaya channeling old Hollywood glamor in a saffron Louis Vuitton gown, Angelina Jolie wearing a dazzling crystal chain McQueen dress and Tilda Swinton donning a custom embroidered Chanel jacket.

Online, such red carpet outfits have long afterlives as they are shared around social media, dissected by influencers and journalists alike. But the real-life fate of the garments themselves is less well-publicized. What happens to them after their moment of fame — where do they go and when are they seen again?

The afterlives of outfits can take many different forms — some are kept in storage, some are displayed at exhibitions, some wend their way onto the open market and are auctioned, and some are bought by the celebrities wearing them. Occasionally, some d

# full code

In [95]:
# CNN full code
import time
import google.generativeai as genai


# model for CNN, summarize
# flash와 pro 양자택일, 어느쪽을 택하든 코드가 크게 달라지진 않음
#system_instruction=
model = genai.GenerativeModel("gemini-1.5-flash")
#model = genai.GenerativeModel("gemini-pro")

# give model an article
chat = model.start_chat(history=[{'role':'user', 'parts':article.content}])

# create summary
start=time.time()
summary=[]
summary[0] = chat.send_message("입력된 내용을 영어로 요약해서 출력해줘. 요약문은 100자 이내여야 해.").text
while len(summary)>400:
    summary[0] = chat.send_message("너무 길어, 더 줄여줘.").text
summary[1] = chat.send_message("요약본을 한국어로 번역해줘").text
end=time.time()
print(f"summary created: {end - start:.5f}sec")

# create vocab
start=time.time()
prompt="""
영문 요약본에서 적당한 난이도의 단어 3개만 찾아줘, 단어 옆에  한국어 뜻을 붙여줘. 단어와 뜻만 출력해.
예시: 
collates 모으다
Crucial 중요한
Attributed 귀속된
"""

response = chat.send_message(prompt)
response=response.text.split('\n')
response.pop()
vocab={}
for i in range(len(response)):
    v,m=response[i].split(maxsplit=1)
    vocab[v]=m
end=time.time()
print(f"vocab created: {end - start:.5f}sec")

summary created: 2.58857sec
vocab created: 0.50424sec


# Summary

In [106]:
#summary 생성 셀
# flash와 pro 양자택일, 어느쪽을 택하든 코드가 크게 달라지진 않음
#system_instruction=
model = genai.GenerativeModel("gemini-1.5-flash")
#model = genai.GenerativeModel("gemini-pro")

# give model an article
chat = model.start_chat(history=[{'role':'user', 'parts':article}])

# create summary
start=time.time()
summary = chat.send_message("입력된 내용을 영어로 요약해서 출력해줘. 요약문은 100자 이내여야 해.").text
while len(summary)>400:
    summary = chat.send_message("너무 길어, 더 줄여줘.").text
summary += chat.send_message("요약본을 한국어로 번역해줘").text
end=time.time()
print(f"summary created: {end - start:.5f}sec")

summary created: 2.18254sec


In [85]:
article.title

'Risks from ash and air pollution remain for people returning after wildfires. Here’s how to stay safe'

In [107]:
print(summary[0])
print(summary[1])

Despite devastating LA wildfires, the Grammys (Feb 2nd) and Oscars (March 2nd) will proceed.  Both ceremonies will shift focus to honor first responders and raise funds for wildfire relief.  The Grammys will incorporate a charitable component, while the Oscars extended voting and postponed its luncheon. Other award shows have been postponed or cancelled due to the fires.
LA 산불에도 불구하고 그래미 어워드(2월 2일)와 아카데미 시상식(3월 2일)은 예정대로 진행됩니다.  두 시상식 모두 소방대원들을 기리고 산불 구호 기금을 모금하는 데 초점을 맞출 것입니다. 그래미 어워드는 자선 활동을 포함하고, 아카데미 시상식은 투표 기간을 연장하고 오찬 행사를 연기했습니다.  다른 시상식들은 산불로 인해 연기되거나 취소되었습니다.



In [108]:
print("원문 길이:", len(article.content))
print("요약문 길이:", len(summary[0]))

원문 길이: 4582
요약문 길이: 373


In [23]:
# 영어 요약문
summary[0]

# 한국어 요약문
summary[1]

'한때 잠깐의 명성 후 잊혀지던 레드카펫 드레스들은 이제 신중하게 계획된 미래를 갖게 되었다. 오스카상이나 메트 갈라 같은 중요한 행사들 덕분에 이 의상들의 가치와 보존이 중요해졌다. 패션 하우스들은 종종 소유권을 유지하며, 드레스들을 보관하거나 전시회에 사용한다.  일부 유명인들은 자신의 드레스를 구매하지만, 레이디 가가처럼 예상치 못하게 경매 시장에 드레스가 등장하는 경우도 있다. 때로는 타일라의 발망 드레스처럼 행사 중간에 드레스가 변형되는 등 극적으로 수명이 끝나는 경우도 있다.'

# 단어 추출(vocab)

In [89]:
# 단어 추출
# flash와 pro 양자택일, 어느쪽을 택하든 코드가 크게 달라지진 않음
model = genai.GenerativeModel("gemini-1.5-flash")
#model = genai.GenerativeModel("gemini-pro")

# give model an article
chat = model.start_chat(history=[{'role':'user', 'parts':article}])

# create vocab
start=time.time()
prompt="""
영문 요약본에서 적당한 난이도의 단어 3개만 찾아줘, 단어 옆에  한국어 뜻을 붙여줘. 단어와 뜻만 출력해. 
예시: 
collates 모으다
Crucial 중요한
Attributed 귀속된
"""

response = chat.send_message(prompt)
response=response.text.split('\n')
response.pop()
vocab={}
for i in range(len(response)):
    v,m=response[i].split(maxsplit=1)
    vocab[v]=m
end=time.time()
print(f"vocab created: {end - start:.5f}sec")

vocab created: 0.58818sec


In [90]:
vocab

{'dissipates': '사라지다', 'off-gas': '방출하다', 'aggravating': '악화시키는'}

In [1]:
chat.history

NameError: name 'chat' is not defined

In [43]:
response

['meticulously 세심하게', 'garnered 얻다', 'daunting 벅찬']